# Breaking Down the Example File to See What It Does

In [5]:
import json
import numpy
import math
import requests
import random
import string
import secrets
import time
import re
import collections
from nltk import ngrams, FreqDist

For the main file, I am working off of another file for a large portion of it. However it is currently not weighting the n-grams properly. I want to go through the original file and see how they do it, but the actual code is super hard for me to read... I want to like hack it to pieces here.

In [6]:
tf = open("words_250000_train.txt","r")
full_dict = tf.read().splitlines()
tf.close()


First the init:

need `build_dictionary()` first

In [7]:
def build_dictionary(dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

In [9]:
tries_remaining = 6
guessed_letters = []

full_dictionary_location = "words_250000_train.txt"
test_dictionary_location = "words_test.txt"
full_dictionary = build_dictionary(full_dictionary_location) 
test_dictionary = build_dictionary(test_dictionary_location)
full_dictionary_common_letter_sorted = collections.Counter("".join(full_dictionary)).most_common()
current_dictionary = []
wins = []
lose_words = []

# building relevant data structures such as substring and individual letter frequencies
f = open("words_250000_train.txt", "r")
df = []
for x in f:
    df.append(x[:-1])

temp = []
_2gram = []
_3gram = []
_4gram = []    
_5gram = []

for word in df:
    temp.append(list(word))         
    _2gram.extend(list(ngrams(word, 2, pad_left=True, pad_right=True)))
    _3gram.extend(list(ngrams(word, 3, pad_left=True, pad_right=True)))
    _4gram.extend(list(ngrams(word, 4, pad_left=True, pad_right=True)))
    _5gram.extend(list(ngrams(word, 5, pad_left=True, pad_right=True)))

freq_2 = FreqDist(_2gram)
freq_3 = FreqDist(_3gram)
freq_4 = FreqDist(_4gram)
freq_5 = FreqDist(_5gram)

freq_2 = [(elem, freq_2.get(elem)) for elem in freq_2]
freq_3 = [(elem, freq_3.get(elem)) for elem in freq_3]
freq_4 = [(elem, freq_4.get(elem)) for elem in freq_4]
freq_5 = [(elem, freq_5.get(elem)) for elem in freq_5]

prev_guess = []

vowels = ['a','e','i','o','u']

word_len_dict = {}
for i in range(3, 30):
    word_len_dict[i] = []
    for words in df:
        if(len(words)>i):
            for j in range(len(words)-i+1):
                word_len_dict[i].append(words[j:j+i]) 

In my code, I think I have all of this... except I don't do the thing where I analyze vowel count.

In [10]:
print(full_dictionary_common_letter_sorted[0:10])

[('e', 233745), ('i', 184746), ('a', 179837), ('n', 152259), ('o', 150052), ('r', 149228), ('s', 148462), ('t', 137277), ('l', 122431), ('c', 89367)]


In [15]:
word_len_dict[28]

['cyclotrimethylenetrinitramin',
 'yclotrimethylenetrinitramine',
 'trinitrophenylmethylnitramin',
 'rinitrophenylmethylnitramine']

`word_len_dict` is all of the words arranged into a dictionary by word length... I guess we will see why this is important later

I have all of the `freq_n` in my code

In [16]:
freq_2[0:10]

[(('s', None), 41635),
 (('e', 'r'), 39200),
 (('i', 'n'), 37762),
 (('e', None), 33665),
 (('t', 'i'), 27596),
 (('o', 'n'), 27389),
 (('e', 's'), 27128),
 (('t', 'e'), 26219),
 (('a', 'n'), 25117),
 (('r', 'e'), 24800)]

In my code, I got rid of the ones with `None` because I think I tried to match stuff a bit differently...

I don't understand how the actual choosing happens really...

In [ ]:
def candsort(cands, invalids, vowels, vowel_ratio):
    
    # chooses the highest scoring (most common) and valid candidate letter from candidate letters
    for cand in cands:
        if cand[0] == None or cand[0] in invalids:
            continue
        if cand[0] in vowels and vowel_ratio > 0.5:
            continue
        return cand 
    return ('!', 0, 1)

This is the main part that matches the n-grams to the individual words.

In [ ]:
def ngram(self, word, index, invalids, freqs, vowel_ratio):
    
    # for each '_' found in the missing word, find valid substring matches and get appropriate weights,
    # giving priority to more complete and longer substrings, tiebreakers between longer substrings are
    # broken by higher weights or frequencies
    
    # candidate tuple structure is: (suggestion, weight, rank) 
    
    freq_2, freq_3, freq_4, freq_5 = freqs
    score1 = ('!', 0, 1)
    score2 = ('!', 0, 1)
    score3 = ('!', 0, 1)
    
    # i. case    
    if index == 0:       
        if word[index+1] == '.':
            return ('!', 0, 1)

        # iXXX Case
        if (len(word) >= 4) and ('.' not in word[index+1:index+4]):
                cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == None) and 
                                                                        (elem[0][2] == word[index+1]) and 
                                                                        (elem[0][3] == word[index+2]) and 
                                                                        (elem[0][4] == word[index+3])]
                return self.candsort(cands, invalids, vowels, vowel_ratio)

        # iXX Case
        if (len(word) >= 3) and ('.' not in word[index+1:index+3]):
                cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == None) and 
                                                                        (elem[0][2] == word[index+1]) and 
                                                                        (elem[0][3] == word[index+2])]
                return self.candsort(cands, invalids, vowels, vowel_ratio)

        # iX case
        cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == None) and 
                                                                (elem[0][2] == word[index+1])]       
        return self.candsort(cands, invalids, vowels, vowel_ratio)


    # .i case    
    if index == len(word)-1:      
        if word[index-1] == '.':
            return ('!', 0, 1)

        # XXXi case:
        if (len(word) >= 4) and ('.' not in word[index-3:index]):
                cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                        (elem[0][1] == word[index-2]) and 
                                                                        (elem[0][2] == word[index-1]) and 
                                                                        (elem[0][4] == None)]
                return self.candsort(cands, invalids, vowels, vowel_ratio)         

        # XXi case
        if (len(word) >= 3) and ('.' not in word[index-2:index]):
                cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                        (elem[0][1] == word[index-1]) and 
                                                                        (elem[0][3] == None)]
                return self.candsort(cands, invalids, vowels, vowel_ratio)    

        # Xi case
        cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                (elem[0][2] == None)]
        return self.candsort(cands, invalids, vowels, vowel_ratio)


    else:  
        # .i. case
        if word[index-1] == '.' and word[index+1] == '.':
            return ('!', 0, 1)


        # .iX family
        if word[index-1] == '.'and word[index+1] != '.': 


            # .iXXXX case
            if (len(word) - index >= 5) and (index >= 1) and ('.' not in word[index+1:index+5]):
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2]) and 
                                                                            (elem[0][4] == word[index+3])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # X.iXX case        
            if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][3] == word[index+1]) and 
                                                                            (elem[0][4] == word[index+2])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # XX.iX case                
            if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index-1]):
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][4] == word[index+1])]
                    score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]                    


            # .iXX case            
            if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                    cands = [(elem[0][0], elem[1], 3) for elem in freq_3 if (elem[0][1] == word[index+1]) and 
                                                                            (elem[0][2] == word[index+2])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # X.iX case
            if (len(word) - index >= 2) and (index >= 2) and (word[index-2] != '.'):
                    cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][3] == word[index+1])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]        

            # .iX        
            cands = [(elem[0][0], elem[1], 2) for elem in freq_2 if elem[0][1] == word[index+1]]
            return self.candsort(cands, invalids, vowels, vowel_ratio)


        # Xi. family
        if word[index-1] != '.'and word[index+1] == '.':

            # XXXXi. case
            if (len(word) - index >= 2) and (index >= 4) and ('.' not in word[index-4:index-1]):
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][2] == word[index-1])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # XXi.X case                    
            if (len(word) - index >= 3) and (index >= 2) and (word[index+2] != '.') and (word[index-2] != '.'):
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][4] == word[index+2])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)
            
            # Xi.XX case
            if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index+2:index+4]):
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+2]) and 
                                                                            (elem[0][4] == word[index+3])]
                    score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]  


            # XXi. case
            if (index >= 2) and (word[index-2] != '.'):
                    cands = [(elem[0][2], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)
            
            # Xi.X case
            if (len(word) - index >= 3) and (index >= 1) and (word[index+2] != '.'):
                    cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+2])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]     

            # Xi. case
            cands = [(elem[0][1], elem[1], 2) for elem in freq_2 if elem[0][0] == word[index-1]]
            return self.candsort(cands, invalids, vowels, vowel_ratio)


        # XiX family
        if word[index-1] != '.'and word[index+1] != '.':
            
            # XXiXX case
            if (len(word) - index >= 3) and (index >= 2) and ('.' not in word[index-2:index+3]):
                    cands = [(elem[0][2], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+1]) and
                                                                            (elem[0][4] == word[index+2])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # XiXXX case
            if (len(word) - index >= 4) and (index >= 1) and ('.' not in word[index-1:index+4]):
                    cands = [(elem[0][1], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2]) and
                                                                            (elem[0][4] == word[index+3])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # XXXiX case
            if (len(word) - index >= 2) and (index >= 3) and ('.' not in word[index-3:index+2]):
                    cands = [(elem[0][3], elem[1], 5) for elem in freq_5 if (elem[0][0] == word[index-3]) and 
                                                                            (elem[0][1] == word[index-2]) and 
                                                                            (elem[0][2] == word[index-1]) and
                                                                            (elem[0][4] == word[index+1])]
                    score3 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if (score1 != score2) or (score2 != score3) or (score1 != score3):
                best_score = sorted([score1, score2, score3], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]                          


            # XiXX case
            if len(word) - index >= 3 and word[index+2] != '.':
                    cands = [(elem[0][1], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-1]) and 
                                                                            (elem[0][2] == word[index+1]) and 
                                                                            (elem[0][3] == word[index+2])]
                    score1 = self.candsort(cands, invalids, vowels, vowel_ratio)

            # XXiX case
            if index >= 2 and word[index-2] != '.':
                    cands = [(elem[0][2], elem[1], 4) for elem in freq_4 if (elem[0][0] == word[index-2]) and 
                                                                            (elem[0][1] == word[index-1]) and 
                                                                            (elem[0][3] == word[index+1])]
                    score2 = self.candsort(cands, invalids, vowels, vowel_ratio)

            if score1 != score2:
                best_score = sorted([score1, score2], key = lambda x: (x[2], x[1]), reverse=True)
                return best_score[0]            


            # XiX case
            cands = [(elem[0][1], elem[1], 3) for elem in freq_3 if (elem[0][0] == word[index-1]) and 
                                                                    (elem[0][2] == word[index+1])]
            return self.candsort(cands, invalids, vowels, vowel_ratio)